In [1]:
import pandas as pd

df = (pd.read_csv("../metadata/merged_metadata_popularity_std.csv"))
df

,Unnamed: 0,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,...,turn_durations_10p,turn_durations_90p,features_embedding_mean,features_embedding_std,mfccs_mean,mfccs_std,ratings_ast,ratings_sum,log_ratings_views_norm,log_ratings_views_norm_cat
0,0,1137,Sharing powerful stories from his anti-obesity...,1313,TED2010,2010-02-20 02:00:00,49,Jamie Oliver,Jamie Oliver: Teach every child about food,1,...,0.6010,6.1600,"[-0.667717317619418, -0.19405687228746657, -0....","[0.2455721431540919, 0.3398221102259164, 0.302...","[-256.6122131347656, 104.05799865722656, -38.1...","[89.68684387207031, 37.565608978271484, 36.178...","[{'id': 23, 'name': 'Jaw-dropping', 'count': 1...",16270,1.108835,high
1,1,209,"In a demo that drew gasps at TED2010, Blaise A...",465,TED2010,2010-02-11 02:00:00,28,Blaise Agüera y Arcas,Blaise Agüera y Arcas: Augmented-reality maps,1,...,0.5610,7.8000,"[-0.8574728478566028, -0.47486242682870144, -0...","[0.2439265217571555, 0.30463898800740546, 0.25...","[-249.42771911621094, 116.88634490966797, -45....","[77.43639373779297, 44.49580383300781, 36.3925...","[{'id': 23, 'name': 'Jaw-dropping', 'count': 8...",2909,0.656105,medium
2,2,250,The leader of Britain's Conservative Party say...,839,TED2010,2010-02-10 02:00:00,29,David Cameron,David Cameron: The next age of government,1,...,0.9730,9.5176,"[-0.6890714909449476, -0.31542495394982295, -0...","[0.2456267015731934, 0.341043095119484, 0.2667...","[-262.04119873046875, 163.34193420410156, -18....","[59.985923767089844, 35.63631820678711, 32.335...","[{'id': 8, 'name': 'Informative', 'count': 233...",1542,1.300880,high
3,3,948,"At TED2010, Bill Gates unveils his vision for ...",1669,TED2010,2010-02-12 02:00:00,38,Bill Gates,Bill Gates: Innovating to zero!,1,...,1.2042,7.2302,"[-0.8741625687103696, -0.1531132806942128, -0....","[0.22519481206931632, 0.34352923686700454, 0.2...","[-322.3145446777344, 107.66773986816406, -21.1...","[87.99868774414062, 44.7060546875, 34.04744338...","[{'id': 21, 'name': 'Unconvincing', 'count': 1...",4178,-0.451176,medium
4,4,277,The land of the free has become a legal minefi...,1101,TED2010,2010-02-13 02:00:00,24,Philip K. Howard,Philip K. Howard: Four ways to fix a broken le...,1,...,0.9420,7.0400,"[-0.9399830150384938, -0.07696298592792386, -0...","[0.23492171793746783, 0.29845273824317015, 0.3...","[-227.3542938232422, 135.99923706054688, -27.0...","[59.40467834472656, 36.28407287597656, 28.0525...","[{'id': 23, 'name': 'Jaw-dropping', 'count': 1...",2236,2.177262,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,1661,18,Could it be wrong to help children in need by ...,791,TEDxSydney,2016-05-24 03:00:00,5,Tara Winkler,Tara Winkler: Why we need to end the era of or...,1,...,1.2862,10.5238,"[-0.5928106715106979, -0.4364868147433903, -1....","[0.18332318386953075, 0.3438698730095391, 0.25...","[-267.6882629394531, 111.9858627319336, -36.98...","[105.70977783203125, 40.21207046508789, 44.026...","[{'id': 10, 'name': 'Inspiring', 'count': 119}...",413,-1.293226,low
1662,1662,3,Digital archaeologist Chance Coughenour is usi...,717,TEDxHamburg,2016-06-08 03:00:00,5,Chance Coughenour,Chance Coughenour: How your pictures can help ...,1,...,0.7803,8.6577,"[-0.6815459389205306, -0.34194393655162353, -0...","[0.2092779610696729, 0.26020847797274077, 0.23...","[-278.66070556640625, 118.9484634399414, -42.9...","[121.16292572021484, 61.404788970947266, 34.15...","[{'id': 9, 'name': 'Ingenious', 'count': 16}, ...",68,-4.461383,low
1663,1663,45,"We all have origin stories and identity myths,...",1156,TEDxExeter,2015-04-24 03:00:00,1,Chetan Bhatt,Chetan Bhatt: Dare to refuse the origin myths ...,1,...,0.8976,8.4156,"[-0.41143028348821165, -0.5835879533895044, -0...","[0.18980604195155124, 0.3083742035890294, 0.31...","[-266.416748046875, 86.22337341308594, -25.184...","[117.83336639404297, 56.634765625, 30.15963172...","[{'id': 9, 'name': 'Ingenious', 'count': 24}

In [15]:
from scripts.generate_dataset_with_aggregations import get_related_result
import tqdm

tasks = ["asr", "emotion", "strength", "positivity"]

def get_asr_annotated(input_json):
    key = list(input_json.keys())[0]

    utterances = {}
    for pred in input_json[key]:
        
        id = int(pred["id"])
        
        if id not in utterances.keys():
            utterances[id] = {}
            
        task = pred["task"]
        if task not in tasks:
            continue
            
        final_label = pred.get('finalLabel')
        
        if final_label is not None:
            utterances[id][task] = final_label
            
    utterances_sorted = [utterances[key] for key in sorted(list(utterances.keys()))]
    
    annotated_transcript = ""
    for utter in utterances_sorted:
        asr = utter['asr']
        behaviors = [k for k in utter.keys() if k != 'asr']
        
        annotations = "".join([f"({utter[behavior]}) " for behavior in behaviors if utter[behavior] != "neutral"])
        annotations = annotations + ": " if annotations != "" else ""
        annotated_transcript += annotations + f"{asr} "
        
    return annotated_transcript
        
            

features_metadata = []

for idx, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    # Modeling api result
    result = get_related_result(row)

    if result is None:
        continue
        
    annotated_transcript = get_asr_annotated(result)
        
    entry = {'url': row['url'], 'transcript': annotated_transcript}
    features_metadata.append(entry)

100%|██████████| 1666/1666 [00:22<00:00, 74.68it/s]


In [13]:
features_metadata_df = pd.DataFrame(features_metadata)
print(f"Length of generated dataset: {len(features_metadata_df)}")
features_metadata_df.to_csv("../metadata/asr_annotated.csv")

Length of generated dataset: 1666


In [23]:
import pandas as pd

df_1 = pd.read_csv("../metadata/asr_annotated.csv")
df_2 = pd.read_csv("../metadata/merged_metadata_popularity_features_std.csv")
df = pd.merge(df_2, df_1, on="url")

In [60]:
metric = "log_Beautiful_norm"
df.sort_values(by=metric, ascending=False)[['url', metric, 'emotion_sad_mean', 'emotion_sad_std', 'pauses_10p', 'strength_weak_90p']]

,url,log_Beautiful_norm,emotion_sad_mean,emotion_sad_std,pauses_10p,strength_weak_90p
302,https://www.ted.com/talks/robert_hammond_build...,2.730322,0.027330,0.020144,0.0540,0.07010
1309,https://www.ted.com/talks/teitur_home_is_a_son...,2.313233,0.253629,0.133639,1.2432,0.55441
1093,https://www.ted.com/talks/daria_van_den_bercke...,2.189228,0.162163,0.061104,0.1600,0.26522
1652,https://www.ted.com/talks/rhiannon_giddens_3_s...,2.154891,0.063914,0.079390,0.0800,0.09728
26,https://www.ted.com/talks/natalie_merchant_sin...,2.144947,0.210726,0.168163,0.4570,0.69900
...,...,...,...,...,...,...
881,https://www.ted.com/talks/rodrigo_canales_the_...,-2.769530,0.034212,0.022394,0.6002,0.16241
680,https://www.ted.com/talks/adam_davidson_what_w...,-2.776681,0.022145,0.015088,0.0540,0.08520
438,https://www.ted.com/talks/stephen_coleman_the_...,-2.872194,0.060598,0.042865,0.0405,0.20059
638,https://www.ted.com/talks/david_pizarro_the_st...,-3.610775,0.052872,0.036812,0.1200,0.14500


In [62]:
df.sort_values(by=metric, ascending=False).loc[1299, "transcript_y"]

"(weak) When I was only three or four, I fell in love with poetry, with the rhythms and the music of language, with the power of metaphor and of imagery. (sad) poetry being the essence of communication, the discipline, the distillation. (sad)(weak) And all these years later, the poems I'll read today are from my just finished seventh book of poetry.  Five years ago, I was diagnosed with Parkinson's disease. Though there's no cure yet, advances in treatment are really impressive. (sad)(weak) But you can imagine that I was appalled to learn that women are largely left out of research trials, despite gender-specific medical findings having demonstrated that we are not actually just small men.  who happen to have different reproductive systems. (weak)Gender specific medicine is good for men too. But you bring to a crisis the person you already are. (sad)(weak) including the, yes, momentum that you've learned to invoke through passionate caring and through action, both of which require but 